In [ ]:
import os
import sys
import ccxt
import pandas as pd
import csv
from datetime import datetime
import time
# import pytz
import json
import numpy as np 
from pymongo import MongoClient
from bson.objectid import ObjectId
from pprint import pprint



print('python', sys.version)
print('CCXT Version:', ccxt.__version__)

In [ ]:
mongo_client = MongoClient()

exchanges = {}
config = False
with open ('config.json') as js: 
        config = json.load(js)

for exkey, exconf in config['exchanges'].items():
        if exconf['enabled'] and not exkey in exchanges:
                excred = exconf['cred']
                for crk, crv in excred.items():
                        if crv[0] == "$" : 
                                excred[crk] = os.getenv(crv[1:])

                exchanges[exkey] = getattr(ccxt, exkey) (excred)

In [ ]:
exchange_id = 'kucoinfutures'
symbol = 'XBTUSDTM'
timeframe = '4h'
coll_name = f'{symbol}-{timeframe}'
collection = mongo_client[exchange_id][coll_name]
df = pd.DataFrame(collection.find())
df.set_index(df["timestamp"], inplace=True, drop=True)
df.drop(columns=['_id', 'timestamp', 'date'], inplace=True)
df

In [ ]:
data_dir = f'./data/{exchange_id}'
if not os.path.exists(data_dir): os.makedirs(data_dir) 

csv_fname = f'{data_dir}/{symbol}-{timeframe}.csv'

df.to_csv(csv_fname,header=False)
# df.to_records()

In [ ]:
# export klines from MongoDB to csv files
for exchange_id in ['kucoinfutures']:
  for coll_name in mongo_client[exchange_id].list_collection_names():
    symbol,timeframe = coll_name.split('-')
    print(f'Exporting {coll_name}...')
    collection = mongo_client[exchange_id][coll_name]
    df = pd.DataFrame(collection.find())
    df.set_index(df["timestamp"], inplace=True, drop=True)
    df.drop(columns=['_id', 'timestamp', 'date'], inplace=True)
    data_dir = f'./data/{exchange_id}'
    if not os.path.exists(data_dir): os.makedirs(data_dir)
    csv_fname = f'{data_dir}/{symbol}-{timeframe}.csv'
    df.to_csv(csv_fname,header=False)
